# Homework 1
###### by [Movsisyan Mher](https://movsisyan.info/?redir=https://movsisyan.info?src=dbandds-hw1)
---

### 1. Relational schema design (5 pts)
In the first session, we ran a survey and collected your responses. The file “Database-Welcome.xlsx” is uploaded to
Moodle, in the current assignment.  
<br>
Use the data from the “Database-Welcome.xlsx” file to define relational schemas.  
Define the schemas in the relational style, similar to the example used in class S = (ID, name, department).  
For every attribute in the schema, specify its domain.  
For each schema specify the primary key that you chose. Explain why it works as a primary key.  

In [1]:
import pandas as pd
import numpy as np

# readin data
df = pd.read_excel('data\Database-Welcome.xlsx')

df.head()

,Date,Session,Voter,Welcome to DS 205!:,What are your interests out of studies?:,Which year of studies are you in?:,Which one describes your knowledge and experience with databases?:,Which programming languages do you know?:,What are your expectations from this course?: 1,Can you bring a laptop computer to the class?:,Thank you:
0,2022-01-20,4,4,Heart,Basketball Drawing_portraits Coding,3rd (Junior),Intermediate,Python Julia,Pass this course\nIf possible learn some funda...,Yes,Heart
1,2022-01-20,4,5,NaN,NaN,2nd (Sophomore),Advanced,Python T-Sql,Gain advanced knowledge of databases from A-Z,Yes,NaN
2,2022-01-20,4,6,Heart,Reading Piano Classical_music,2nd (Sophomore),Beginner,Python R JS,NaN,Yes,Heart
3,2022-01-20,4,7,NaN,Playing_the_guitar Hiking,2nd (Sophomore),Beginner,Python JS,NaN,Yes,NaN
4,2022-01-20,4,8,NaN,NaN,NaN,Beginner,Python CSharp,NaN,Yes,Heart


Dropping unnecessary columns

In [2]:
df.drop(["Date", "Session", "Welcome to DS 205!:", "Thank you:"], axis=1, inplace=True)
df.head()

,Voter,What are your interests out of studies?:,Which year of studies are you in?:,Which one describes your knowledge and experience with databases?:,Which programming languages do you know?:,What are your expectations from this course?: 1,Can you bring a laptop computer to the class?:
0,4,Basketball Drawing_portraits Coding,3rd (Junior),Intermediate,Python Julia,Pass this course\nIf possible learn some funda...,Yes
1,5,NaN,2nd (Sophomore),Advanced,Python T-Sql,Gain advanced knowledge of databases from A-Z,Yes
2,6,Reading Piano Classical_music,2nd (Sophomore),Beginner,Python R JS,NaN,Yes
3,7,Playing_the_guitar Hiking,2nd (Sophomore),Beginner,Python JS,NaN,Yes
4,8,NaN,NaN,Beginner,Python CSharp,NaN,Yes


Defining a schema for the survey results:  
<br>
$$ Voters = (ID, yearOfStudy, priori, laptop) $$
Voters.year_of_study is not timeless, but I'll ignore that. It is the "Which year of studies are you in?:" column.  
Voters.ID is a primary key, it is the "Voter" column in the data because it is unique and an obvious choice.  
Voters.priori is the level of knowledge and experience of the voter, is the "Which one describes your knowledge and experience with databases?:" column in the data.  
Voters.laptop is the "Can you bring a laptop computer to the class?:" column in the data.
<br>
$$ Interests = (ID, voter, interest) $$  
Interests.ID is a primary key, it is generated from the segmentation process.  
Interests.voter is a foreign key that references Voters.ID
<br>
$$ LanguageKnowers = (ID, voter, language) $$
LanguageKnowers.ID is a primary key, it is generated from the segmentation process.  
LanguageKnowers.voter is a foreign key that references Voters.ID  
LanguageKnowers.language is the individual cleaned-up language from the "Which programming languages do you know?:" column in the data

### 2. Data clean up (4 pts)
Think of a method to identify the data items that have the same meaning. For example, JS and JavaScript and Java script have the same semantics, however written in slightly different ways. Describe your approach of solving this issue in simple words.  
Design a function that receives a token (e.g. JS) as the input and returns an output which is unique for all the variants.  
<br>
For example, given the function f(x) it should require that:  
$$ f (“JavaScript”) = f (“JavaScript”) = f (“JS”) = “JS” $$
and  
$$ f (“py”) = f (“P ython”) = f (“python”) = “Python” $$
and so on.  
Implement the body of the function in your preferred language, or using pseudo-code.

In [3]:
# looking at the column in interest
df.iloc[:, 4]

0                  Python Julia
1                  Python T-Sql
2                   Python R JS
3                     Python JS
4                 Python CSharp
5     Python C_sharp Javascript
6                      Python R
7           Javascript Python R
8                   python r js
9                      python R
10                       Python
11                          NaN
12                     Python R
13                R Python Java
14          Python R Javascript
15            Python Javascript
16                          NaN
17                       python
18                          NaN
Name: Which programming languages do you know?:, dtype: object

In [4]:
# getting (almost) unique entries for the known languages
p_lang_entries = [[*i.split(' ')] for i in df.iloc[:, 4].fillna('None')]
p_langs = pd.Series([x for l in p_lang_entries for x in l]).str.lower()
p_langs.value_counts()

python        16
r              8
javascript     4
js             3
none           3
julia          1
t-sql          1
csharp         1
c_sharp        1
java           1
dtype: int64

In [5]:
from fuzzywuzzy import fuzz
import warnings

warnings.filterwarnings("ignore")

# creating a fuzz-ratio matrix of already known languages
lang_fuzz = pd.DataFrame(columns = p_langs.unique(), index = p_langs.unique()).fillna(0)

# filling the matrix with the fuzzy-ratio values
for i, col in lang_fuzz.iteritems():
    for j, row in col.iteritems():
        lang_fuzz.loc[i, j] = fuzz.ratio(i, j)

lang_fuzz

C:\Users\zolobrine\AppData\Roaming\Python\Python310\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


,python,julia,t-sql,r,js,csharp,c_sharp,javascript,none,java
python,100,0,18,0,0,17,15,25,40,0
julia,0,100,20,0,29,18,17,27,0,44
t-sql,18,20,100,0,29,18,17,13,0,0
r,0,0,0,100,0,29,25,18,0,0
js,0,29,29,0,100,25,22,33,0,33
csharp,17,18,18,29,25,100,92,38,0,20
c_sharp,15,17,17,25,22,92,100,35,0,18
javascript,25,27,13,18,33,38,35,100,0,57
none,40,0,0,0,0,0,0,0,100,0
java,0,44,0,0,33,20,18,57,0,100


Writing the function

In [6]:
languages = [
    "python", "csharp", "julia", 
    "none", "java", "javascript", 
    "r", "scala", "matlab", "go", 
    "c", "c++", "ruby", "perl", "py",
    "php", "swift", "haskell", "js","lua", 
    "rust", "visual_basic", "sql", # gonna treat t-sql, postgresql, mysql as the same
    "assembly", "ecmascript", "c#"
    ]

def match_lang(x, threshold = 74, known = languages):
    """Matches a string to a given programming language. If it fails 
    to do so, it creates a new entry in the list of languages"""
    
    # making it lowercase
    lower_x = x.lower()
    # creating a dict that will store the fuzzy ratios
    scores = {i:0 for i in known}
    
    # checking fuzzy ratio for each registered language
    for lang in known:
        scores[lang] = fuzz.ratio(lang, lower_x)
    
    # print("Best match:", max(scores, key=scores.get), max(scores.values()))
    
    if max(scores.values()) < threshold:
        print("Adding new language:", x)
        known.append(x)
        # insert new language into the languages table
        
        return x
    else:
        # get best match
        best_x = max(scores, key=scores.get)
        
        # check if it is a known short form
        if best_x == "js" or best_x == "ecmascript":
            return "javascript"
        elif best_x == "py":
            return "python"
        elif best_x == "c#":
            return "csharp"
        
        # return the best match
        return best_x

In [7]:
match_lang("r")

'r'

In [8]:
match_lang("js")

'javascript'

In [9]:
match_lang("jaAVASCRIPT")

'javascript'

In [10]:
match_lang("C#")

'csharp'

In [11]:
match_lang("py")

'python'

### 3. Data population (2 pts)
Visualize the schema instances as tables and populate the data from the Excel file into the tables as rows, so that
they obey the relational schema format

In [25]:
languages = []

df.fillna('None', inplace=True)
lang_id = 0

for voter in df.iterrows():
    for lang in voter[1][4].split(" "):
        match = match_lang(lang)
        if match != "none":
            languages.append((lang_id, voter[1][0], match))
            lang_id += 1

In [26]:
languages

[(0, 4, 'python'),
 (1, 4, 'julia'),
 (2, 5, 'python'),
 (3, 5, 'sql'),
 (4, 6, 'python'),
 (5, 6, 'r'),
 (6, 6, 'javascript'),
 (7, 7, 'python'),
 (8, 7, 'javascript'),
 (9, 8, 'python'),
 (10, 8, 'csharp'),
 (11, 9, 'python'),
 (12, 9, 'csharp'),
 (13, 9, 'javascript'),
 (14, 10, 'python'),
 (15, 10, 'r'),
 (16, 11, 'javascript'),
 (17, 11, 'python'),
 (18, 11, 'r'),
 (19, 12, 'python'),
 (20, 12, 'r'),
 (21, 12, 'javascript'),
 (22, 13, 'python'),
 (23, 13, 'r'),
 (24, 14, 'python'),
 (25, 16, 'python'),
 (26, 16, 'r'),
 (27, 17, 'r'),
 (28, 17, 'python'),
 (29, 17, 'java'),
 (30, 18, 'python'),
 (31, 18, 'r'),
 (32, 18, 'javascript'),
 (33, 19, 'python'),
 (34, 19, 'javascript'),
 (35, 21, 'python')]